In [38]:
import math, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [39]:
df = pd.read_csv("taetfp.csv", encoding='big5')

In [40]:
df["代碼"]='00'+df["代碼"].apply(str)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19665 entries, 0 to 19664
Data columns (total 8 columns):
代碼         19665 non-null object
日期         19665 non-null int64
中文簡稱       19665 non-null object
開盤價(元)     19665 non-null float64
最高價(元)     19665 non-null float64
最低價(元)     19665 non-null float64
收盤價(元)     19665 non-null float64
成交張數(張)    19665 non-null object
dtypes: float64(4), int64(1), object(3)
memory usage: 1.2+ MB


In [42]:
df.shape

(19665, 8)

# Using Linear Regression

In [43]:
forecast_out = int(5) 

In [44]:
df_submit = pd.DataFrame(columns=['ETFid','Mon_ud','Mon_cprice','Tue_ud','Tue_cprice','Wed_ud','Wed_cprice',
                                  'Thu_ud','Thu_cprice','Fri_ud','Fri_cprice'])

In [45]:
symbols = list(set(df["代碼"]))

In [46]:
for i in range(0, 18):
    symbol = symbols[i]
    df2 = df[df["代碼"] == symbols[i]]
    df2['Prediction'] = df2[['收盤價(元)']].shift(-forecast_out) 
    X = np.array(df2.drop(["Prediction","中文簡稱","代碼","成交張數(張)"], 1))
    X = preprocessing.scale(X)
    X_forecast = X[-forecast_out:] 
    X = X[:-forecast_out] 
    y = np.array(df2['Prediction'])
    y = y[:-forecast_out]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    #clf = linear_model.LinearRegression()
    clf = SVR(C=1,kernel='rbf', epsilon=0.2)    
    clf.fit(X_train,y_train)    
    confidence = clf.score(X_test, y_test) 
    #print("confidence: ", confidence)    
    forecast_prediction = clf.predict(X_forecast)
    #print(forecast_prediction)
    
    ud1=1
    ud2=1
    ud3=1
    ud4=1
    ud5=1
    if forecast_prediction[0] - y[len(y)-1] < 0: ud1= -1
    if forecast_prediction[1] - forecast_prediction[0] < 0: ud2= -1
    if forecast_prediction[2] - forecast_prediction[1] < 0: ud3= -1
    if forecast_prediction[3] - forecast_prediction[2] < 0: ud4= -1
    if forecast_prediction[4] - forecast_prediction[3] < 0: ud5= -1    
    
    df_temp =  pd.DataFrame([[symbol,ud1,forecast_prediction[0],ud2,forecast_prediction[1],ud3,forecast_prediction[2],
                          ud4,forecast_prediction[3],ud5,forecast_prediction[4]]],
                         columns=['ETFid','Mon_ud','Mon_cprice','Tue_ud','Tue_cprice','Wed_ud','Wed_cprice',
                                  'Thu_ud','Thu_cprice','Fri_ud','Fri_cprice'])
    df_submit=df_submit.append(df_temp)

C:\Users\Anson\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [47]:
df_submit=df_submit.sort_values(by=['ETFid'])

In [48]:
df_submit.to_csv('ETF_predict_SVR.csv', index=False)